In [22]:
import os
import redis
import hashlib
from metaflow import FlowSpec, step, S3
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle

# Ejemplo de predicción de modelo usando Redis

Este notebook muestra cómo obtener predicciones de un modelo que produce predicciones en lotes. Las predicciones se cargaron en Redis. 

En este notebook, recuperamos las predicciones del modelo almacenadas en Redis. Los datos deben ser leídos, convertidos a cadenas y luego hasheados. Con este enfoque, podemos verificar si los datos existen en Redis y recuperar la predicción correspondiente. En caso de que los datos no existan, asignamos un valor de cero. Esta estrategia simula cómo podría comportarse un servicio en producción ante casos no contemplados.

La ventaja de utilizar Redis en este contexto radica en su capacidad para almacenar datos de forma eficiente en memoria, lo que permite un acceso rápido a las predicciones previamente calculadas. Además, Redis ofrece funcionalidades de almacenamiento en caché y persistencia de datos, lo que garantiza la disponibilidad y la integridad de las predicciones incluso en entornos de producción de alta demanda.

In [23]:
# Conectamos al servidor redis (asegúrate de que el docker compose esté corriendo)
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Configuración de las credenciales de acceso a AWS S3 (minio)
os.environ['AWS_ACCESS_KEY_ID'] = "minio"
os.environ['AWS_SECRET_ACCESS_KEY'] = "minio123"
os.environ['AWS_ENDPOINT_URL_S3'] = "http://localhost:9000"

In [24]:
# Obtención de datos para prueba
df = pd.read_csv("./breast_cancer.csv", header=None)

# Sampleamos 100 valores al azar
df_temp = df.sample(50)

test_values = df_temp.values.tolist()

s3 = S3(s3root="s3://amqtp/")
scaler_obj = s3.get("scaler.pkl")
with open(scaler_obj.path, 'rb') as f:
    scaler = pickle.load(f)

# Aplicamos el scaler a los datos
scaled_values = scaler.transform(df_temp)


# Conversión de valores a cadenas y hash
# Esto debería implementarse en el pipeline. Dado que los números de punto flotante pueden 
# presentar problemas debido a pequeñas variaciones, se podría considerar redondearlos.
string_values = [' '.join(map(str, sublist)) for sublist in scaled_values]
hashed_values = [hashlib.sha256(substring.encode()).hexdigest() for substring in string_values]

# Inicializamos un diccionario para almacenar las salidas del modelo
model_outputs = {}

# Obtenemos las predicciones almacenadas en Redis
for hash_key in hashed_values:
    model_outputs[hash_key] = r.hgetall(f"predictions:{hash_key}")

[[ 0.19367513 -0.52727219  0.10856461 ... -0.53489058 -0.57519089
  -0.87263018]
 [-0.6898559  -1.2210605  -0.74266603 ... -0.48489468  0.3502624
  -0.95207756]
 [-0.20561293 -0.20413791 -0.2331643  ... -0.54429419 -0.32936736
  -1.0006605 ]
 ...
 [-1.13728507  0.10236446 -1.13468556 ... -1.43434649 -0.99775029
  -0.92121311]
 [-0.33870895  0.04534076 -0.31890569 ... -0.23914684  1.14557382
   0.46425782]
 [ 0.29562102  2.72307851  0.45936232 ...  1.90832569  1.93767187
   3.27863853]]


/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Veamos la salida del modelo para diferentes entradas:

In [25]:
print("Salidas de los modelos para las primeras 5 entradas:")
for index, test_value in enumerate(test_values):
    hash_key = hashed_values[index]
    tree_prediction = model_outputs[hash_key].get('tree', 'No disponible')
    svc_prediction = model_outputs[hash_key].get('svc', 'No disponible')
    knn_prediction = model_outputs[hash_key].get('knn', 'No disponible')
    reglog_prediction = model_outputs[hash_key].get('reglog', 'No disponible')
    
    print(f"\nPara la entrada: {test_value}")
    print(f"El modelo tree predice: {tree_prediction}")
    print(f"El modelo svc predice: {svc_prediction}")
    print(f"El modelo knn predice: {knn_prediction}")
    print(f"El modelo reglog predice: {reglog_prediction}")

print("\nSe han mostrado las predicciones para las primeras 5 entradas.")

Salidas de los modelos para las primeras 5 entradas:

Para la entrada: ['14.86', '16.94', '94.89', '673.7', '0.08924', '0.07074', '0.03346', '0.02877', '0.1573', '0.05703', '0.3028', '0.6683', '1.612', '23.92', '0.005756', '0.01665', '0.01461', '0.008281', '0.01551', '0.002168', '16.31', '20.54', '102.3', '777.5', '0.1218', '0.155', '0.122', '0.07971', '0.2525', '0.06827']
El modelo tree predice: Benigno
El modelo svc predice: Benigno
El modelo knn predice: Benigno
El modelo reglog predice: Benigno

Para la entrada: ['11.74', '14.02', '74.24', '427.3', '0.07813', '0.0434', '0.02245', '0.02763', '0.2101', '0.06113', '0.5619', '1.268', '3.717', '37.83', '0.008034', '0.01442', '0.01514', '0.01846', '0.02921', '0.002005', '13.31', '18.26', '84.7', '533.7', '0.1036', '0.085', '0.06735', '0.0829', '0.3101', '0.06688']
El modelo tree predice: Benigno
El modelo svc predice: Benigno
El modelo knn predice: Benigno
El modelo reglog predice: Benigno

Para la entrada: ['13.45', '18.3', '86.6', '555